#Zadanie 2 (7 pkt)
Celem zadania jest rozwiązanie zadania optymalizacji przy pomocy algorytmu genetycznego. Należy znaleźć minimum zadanej funkcji:
\begin{equation}
f(x) = 1.5-\exp \left\{-x_1^2-x_2^2\right\}-0.5 \exp \left\{-\left(x_1-1\right)^2-\left(x_2+2\right)^2\right\}
\end{equation}
Należy wykorzystać mechanizmy krzyżowania punktowego, mutacji oraz selekcji turniejowej. Proszę wykorzystać przygotowany szkielet i zaimplementować niezbędne metody. Opracowane oprogramowanie powinno być uniwersalne - wymiar funkcji, której minimum szukamy może być dowolny (mechanizm *args). Punktacja wygląda następująco:

*   Stworzenie obiektu klasy *Chromosome* z polem *array*, które jest wektorem aktywnych i nieaktywnych genów - **0.5 pkt**
*   Implementacja metody *decode*, która dekoduje część chromosomu (określoną przez początek (*lower_bound*) i koniec (*upper_bound*)) do postaci liczby rzeczywistej. *aoi* to zakres wartości zdekodowanej liczby rzeczywistej. Przykład: liczba 135 w postaci binarnej zapisana przy użyciu 8 bitów to 1000 0111, jeśli nasze *aoi* to [0, 1], to 135 sprowadzone do tego zakresu to 0.529. Można skorzystać z funkcji pomocniczej *min_max_norm* - **1 pkt**
*   Implementacja metody *mutation*, która przyjmuje jako argument prawdopodobieństo mutacji i zmienia wartość jedego, losowego genu na przeciwną - **0.5 pkt**
*   Implementacja metody *crossover*, która wykonuje operację krzyżowania jednopunktowego - **1 pkt**
*   Implementacja metody *eval_objective_func*, która dekoduje cały chromosom (otrzymuje się argumenty funkcji) oraz zwraca wartość funkcji celu dla tych argumentów - **1 pkt**
*   Implementacja metody *tournament_selection*, która przeprowadza selekcję turniejową - **1 pkt**
*   Implementacja metody *reproduce*, która generuje nowe pokolenie - z pewnym prawdopodobieństwem przeprowadza krzyżowanie jednopunktowe lub "przerzuca" rodziców do nowego pokolenia - **0.5 pkt**
*   Implementacja metody *run*, która wykonuje cały alorytm genetyczny dla określonej liczby pokoleń. W każdym pokoleniu należy zapisać dane osobnika z najlepszym chromosomem zdekodowane wartości x i y oraz wartość funkcji strat dla tego osobnika - **0.5 pkt**
*   Proszę, podobnie jak miało to miejsce w przypadku metody gradientowej z poprzednich zajęć, wygenerować wykres przy użyciu funkcji *plot_func* (w przypadku innego typu argumentu *trace*, funkcję można zmodyfikować. Wykres powinien przedstawiać funkcję, której minimum poszukujemy oraz punkty odpowiadające najlepszym osobnikom w danych generacjach, których kolor jest coraz jaśniejszy wraz ze zbliżaniem się do minimum. Proszę zapisać swoje wnioski, w szczególności w odniesieniu do metody gradientowej. - **1 pkt**


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import sys

In [ ]:
def min_max_norm(val, min_val, max_val, new_min, new_max):
  return (val - min_val) * (new_max - new_min) / (max_val - min_val) + new_min

def decimal_value_from_array(array:np.array) -> int:
  value = 0
  index = 0
  # print(np.flip(array))
  for byte in np.flip(array):
    value += byte * (2**index)
    index+=1
  return value

def bit_negation(bit:int) -> int:
  if bit == 0:
    return 1
  return 0

def g_x(x1:float, x2:float) -> float:
  return 1.5-np.exp(-x1**(2)-x2**(2))-0.5*np.exp(-(x1-1)**(2)-(x2+2)**(2))


In [ ]:
class Chromosome:
  def __init__(self, length, array=None): #if array is None it should be initialized with random binary vector
    self.length = length
    self.array = array

    random_gens_list = [random.randint(0,1) for gen in range(self.length)]
    if self.array is None:
      random_gens_list = [random.randint(0,1) for gen in range(self.length)]
      self.array = np.array(random_gens_list)
    assert len(self.array) == self.length, "Array's length is not equal to length"

  def decode(self, lower_bound:int, upper_bound:int, aoi) -> float:
    array_to_decode = self.array[lower_bound : upper_bound]
    # print(array_to_decode)
    max_val = decimal_value_from_array(np.ones((len(array_to_decode)), dtype=int))
    new_min, new_max = aoi
    # print("decimal_value_from_array(self.array)", decimal_value_from_array(array_to_decode))
    # print(" 0, max_val, new_min, new_max",  0, max_val, new_min, new_max)
    # print(" 0, max_val, new_min, new_max", 0, max_val, new_min, new_max)

    return min_max_norm(decimal_value_from_array(array_to_decode), 0, max_val, new_min, new_max)


  def mutation(self, probability):
    if random.random() < probability:
      gen_to_modify = random.randint(0,self.length-1)
      self.array[gen_to_modify] = bit_negation(self.array[gen_to_modify])


  def crossover(self, chromosome : 'Chromosome') -> 'Chromosome':
    assert isinstance(chromosome, Chromosome), "Chromosome object is required as an argument"
    random_index = random.randint(0,len(self.array))
    chromosome1 = np.concatenate((self.array[:random_index], chromosome.array[random_index:]))
    chromosome2 = np.concatenate((chromosome.array[:random_index], self.array[random_index:]))
    self.array = chromosome1
    chromosome.array = chromosome2
    return chromosome

In [ ]:
class GeneticAlgorithm:
  def __init__(self, chromosome_length, obj_func_num_args: int, objective_function, aoi, population_size=1000,
               tournament_size=2, mutation_probability=0.05, crossover_probability=0.8, num_steps=30):
    assert chromosome_length % obj_func_num_args == 0, "Number of bits for each argument should be equal"
    self.chromosome_lengths = chromosome_length
    self.obj_func_num_args = obj_func_num_args
    self.bits_per_arg = int(chromosome_length / obj_func_num_args)
    self.objective_function = objective_function
    self.aoi = aoi
    self.tournament_size = tournament_size
    self.mutation_probability = mutation_probability
    self.crossover_probability = crossover_probability
    self.num_steps = num_steps
    self.population_size = population_size
    self.population = None

  def obtain_args_list(self, chromosome) -> list[float]:
    arg_indx_head = 0
    arg_indx_tail = self.bits_per_arg
    decoded_arguments = []
    for _ in range(self.obj_func_num_args):
      decoded_arguments.append(chromosome.decode(arg_indx_head,arg_indx_tail,self.aoi))
      arg_indx_head += self.bits_per_arg
      arg_indx_tail += self.bits_per_arg
    # print(decoded_arguments)
    return decoded_arguments

  def eval_objective_func(self, chromosome) -> float:
    decoded_arguments = self.obtain_args_list(chromosome)
    return self.objective_function(*decoded_arguments)

  def best_from_individuals(self, rivals: list[Chromosome] = None) -> Chromosome:
    if rivals is None: rivals = self.population
    fittness_of_individuals = [self.eval_objective_func(chr) for chr in rivals]
    #print("min(fittness_of_individuals)", min(fittness_of_individuals))
    #print("max(fittness_of_individuals)", max(fittness_of_individuals))

    best_individual_index = fittness_of_individuals.index(min(fittness_of_individuals))
    return rivals[best_individual_index]


  def tournament_selection(self):
    new_generation = []
    for _ in range(self.population_size):
        tournament_list = [self.population[individual_index] for individual_index in random.sample(range(0, self.population_size), self.tournament_size)] # list of random rivals
        new_generation.append(self.best_from_individuals(tournament_list))
    # print("New generation len():",len(new_generation))
    print(".", end="")
    self.population = new_generation

  def reproduce(self, parents=None):
    # żle zrobione, prawdopodobieństwa to nie fittnes wręcz przeciwnie
    fittness_of_individuals = [self.eval_objective_func(individ) for individ in self.population]
    self.population = random.choices(self.population, weights=fittness_of_individuals, k=self.population_size)

  def plot_func(self, trace):
    X = np.arange(-2, 3, 0.05)
    Y = np.arange(-4, 2, 0.05)
    X, Y = np.meshgrid(X, Y)
    Z = 1.5 - np.exp(-X ** (2) - Y ** (2)) - 0.5 * np.exp(-(X - 1) ** (2) - (Y + 2) ** (2))
    plt.figure()
    plt.contour(X, Y, Z, 10)
    cmaps = [[ii / len(trace), 0, 0] for ii in range(len(trace))]
    plt.scatter([x[0] for x in trace], [x[1] for x in trace], c=cmaps)
    plt.show()

  def run(self):
    # Initialization
    self.population = [Chromosome(self.chromosome_lengths) for _ in range(self.population_size)] # self.population -> List[Chromosomes]

    # best of first
    coordinates_of_best_indiv = self.obtain_args_list(self.best_from_individuals())
    trace = np.array([[*coordinates_of_best_indiv]])
    values_of_func = [self.objective_function(*coordinates_of_best_indiv)]


    # Evolution
    for _ in range(self.num_steps):

      # Reproduction
      # self.reproduce()

      # Selection
      self.tournament_selection()

      # Genetic operators
      temp_generation = []

      # crossover method 1
      assert self.population_size % 2 == 0, "Population has to be even"

      for _ in range(int(self.population_size/2)):
        parent1 = self.population.pop(random.randint(0, len(self.population)-1))
        parent2 = self.population.pop(random.randint(0, len(self.population)-1))
        if random.random() < self.crossover_probability:
          parent2 = parent1.crossover(parent2)

        parent1.mutation(self.mutation_probability)
        parent2.mutation(self.mutation_probability)

        temp_generation.extend((parent1, parent2))
      self.population = temp_generation
      """

      for individual in self.population:
        # Crossover method 2
        if random.random() < self.crossover_probability:
          individual.crossover(self.population[random.randint(0,self.population_size-1)]) # crossover with random individual as a 2-nd parent
        # Mutation
        individual.mutation(self.mutation_probability)
        temp_generation.append(individual)
      self.population = temp_generation

      # best of individuals
      """


      coordinates_of_best_indiv = self.obtain_args_list(self.best_from_individuals())
      trace = np.append(trace, np.array([[*coordinates_of_best_indiv]]), axis=0)
      values_of_func.append(self.objective_function(*coordinates_of_best_indiv))

      print("x,y: ",self.obtain_args_list(self.best_from_individuals()))
      print("f(x,y)",self.objective_function(*coordinates_of_best_indiv))

    #print(trace)
    #print(values_of_func)
    # Plot trace
    self.plot_func(trace)

In [290]:
ga = GeneticAlgorithm(200, 2, g_x, [-1,1])
ga.run()

# Pytania


Jak działa ten algorytm krok po kroku
czy jest tak:
1. inicjacja populacji np. 10000
2. turniejowo bierzemy sobie losowe dwójki z powtarzaniem (lub bez, ale co wtedy bo zostanie 500) i dostajemy nową generację
  * czy jeszcze inaczej z jakimś prawdopodobienstwem tego, że zostanie wybrany
3. crossovery, jak wybrać  rodziców do tego:
  * Czy któraś z moich metod crossover jest poprawna?
  1. dwójkami łączymy wszystkich w pary
  2. dla każdego szukamy randomowego sąsiada i jest tylko jedno dziecko z 2 gości
  3. czy w ogóle jakoś może selekcję turniejową tu albo jakieś prawdopodobieństwa
4. mutacja
5. znajdujemy najlepszego i kolejna generacja od 2 pkt.

